**COSC 44323- Emerging Technologies in Information Technology**

**Mini Project 03 - English to Sinhala Translation System using Transformer Neural Networks**

**Student Number: PS/2019/081**

**Student Name: K. G. M. Pinsara**


---









#Importing Relevant Libraries

#Preparing Data

In [2]:
import random
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

##Mounting the Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Reading the Data File

In [4]:
text_file = "/content/drive/My Drive/Colab_Data_Files/EngSin.txt"
with open(text_file) as f:  # Opening the file in read mode
    lines = f.read().split("\n")[:-1]   # Reading the lines from the file and splitting by newline character

In [5]:
# Print the first 20 lines of the file
i = 0
for line in lines:
  print(line)
  i = i + 1
  if(i==20):
    break

you will receive a package in the mail	ඔයාට තැපෑලෙන් පැකේජ් එකක් හම්බවේවි
you will receive a confirmation code after completing the registration	ලියාපදිංචිය සම්පූර්ණ කලාට පස්සෙ ඔයාට තහවුරු කිරීමේ කේතයක්  හම්බවේවි
you will receive a discount on your next purchase	ඊලඟ මිලදී ගැනීම කරන කොට ඔයාට වට්ටමක් හම්බවේවි
you will receive a phone call from our customer service team	ඔයාට අපේ පාරිභෝගික සේවා කණ්ඩායමෙන් දුරකථන ඇමතුමක් හම්බවේවි
you will receive a notification when your order is ready for pickup	ඔයාගේ ඇණවුම සූදානම් උනාට පස්සෙ ඔයාට දැනුම් දීමක් හම්බවේවි
you will receive a response to your inquiry within 24 hours	ඔයාට පැය 24ක් ඇතුලත ඔයාගෙ විමසීමට පිළිතුරක්  හම්බවේවි
you will receive a gift for your loyalty	ඔයාට ඔයාගේ පක්ෂපාතිත්වය වෙනුවෙන් තෑග්ගක් හම්බවේවි
you will receive an invitation to the event	ඔයාට උත්සවයට ආරාධනාවක් හම්බවේවි
you will receive a refund for the returned item	ඔයාට රිටන් කරන භාණ්ඩය වෙනුවෙන් ආපසු මුදල් ගෙවීමක් හම්බවේවි
you'll receive an email with the details	ඔයාට විස්තර එක්ක

In [6]:
# Print the last 10 lines of the file
for x in range(len(lines)-10,len(lines)):
  print(lines[x])

Once upon a time in a small village there lived a young girl named Lily who had a special gift for growing the most beautiful flowers	එක් කලෙක කුඩා ගමක ලිලී නම් තරුණියක් ජීවත් වූ අතර ඇයට ලස්සනම මල් වගා කිරීම සඳහා විශේෂ තෑග්ගක් තිබුණි
In a distant land a brave knight named Sir Arthur embarked on a perilous journey to rescue a kidnapped princess from an evil sorcerer	දුර රටක ශ්‍රීමත් ආතර් නම් නිර්භීත නයිට්වරයා නපුරු මායාකාරයෙකුගෙන් පැහැරගෙන ගිය කුමරියක් බේරා ගැනීමට භයානක ගමනක් ආරම්භ කළේය
Emily a curious explorer set out on an adventure to uncover the hidden treasures of an ancient lost city deep in the Amazon rainforest	කුතුහලය දනවන ගවේෂකයෙකු වන එමිලි ඇමසන් වනාන්තරයේ ගැඹුරින් ගිලිහී ගිය පැරණි නගරයක සැඟවුණු වස්තු සොයා ගැනීමට ත්‍රාසජනක ගමනක් ආරම්භ කළාය
In the peaceful town of Willowbrook a mischievous cat named Oliver had a talent for getting into amusing and unexpected predicaments	සාමකාමී නගරයක් වන විලෝබෲක්හි ඔලිවර් නම් දඟකාර බළලාට විනෝදජනක සහ අනපේක්ෂිත දුෂ්කරතාවන්ට පත්වීමේ දක්ෂතාවයක් ති

##Spliting the English and Sinhala Translation Pairs

In [7]:
text_pairs = []
for line in lines:
    english, sinhala = line.split("\t") # Split the line by tab to separate English and Sinhala translations
    sinhala = "[start] " + sinhala + " [end]" # Add start and end tokens to the Sinhala translation
    text_pairs.append((english, sinhala))  # Append the English-Sinhalah pair to text_pairs list


In [8]:
# Print 3 randomly chosen pairs
for i in range(3):
    print(random.choice(text_pairs))

('where are you going', '[start] කොහෙද උඹ යන්නේ    [end]')
("i'm very sorry", '[start] ම\u200cට සමාවෙන්න [end]')
('this is real food', '[start] මේ ඇත්ත කෑම [end]')


## Randomizing the Data

In [9]:
import random
random.shuffle(text_pairs)

## Spliting the Data into Training, Validation and Testing

In [10]:
num_val_samples = int(0.15 * len(text_pairs))  # Calculate the number of validation samples
num_train_samples = len(text_pairs) - 2 * num_val_samples  # Calculate the number of training samples
train_pairs = text_pairs[:num_train_samples]  # Assign the first part of shuffled pairs to training set
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]  # Assign the next part to validation set
test_pairs = text_pairs[num_train_samples + num_val_samples:]  # Assign the rest to testing set

In [11]:
# Print sizes of each set
print("Total sentences:",len(text_pairs))
print("Training set size:",len(train_pairs))
print("Validation set size:",len(val_pairs))
print("Testing set size:",len(test_pairs))

Total sentences: 80684
Training set size: 56480
Validation set size: 12102
Testing set size: 12102


In [12]:
len(train_pairs)+len(val_pairs)+len(test_pairs)

80684

##Removing Punctuations

In [13]:
strip_chars = string.punctuation + "¿"  # Define a string containing punctuation marks and "¿"
strip_chars = strip_chars.replace("[", "")  # Remove "[" character from strip_chars
strip_chars = strip_chars.replace("]", "")  # Remove "]" character from strip_chars

# Print regex pattern for stripping punctuation marks
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

## Vectorizing the English and Sinhala Test Pairs

In [14]:
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)  # Convert input string to lowercase
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")  # Remove punctuation marks from the string

vocab_size = 15000  # Define the vocabulary size
sequence_length = 20  # Define the sequence length

# Initialize TextVectorization layers for source (English) and target (Sinhala) texts
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

# Extract English and Sinhala texts from train_pairs
train_english_texts = [pair[0] for pair in train_pairs]
train_sinhala_texts = [pair[1] for pair in train_pairs]

# Adapt the source vectorization layer to the English texts
source_vectorization.adapt(train_english_texts)
# Adapt the target vectorization layer to the Sinhala texts.
target_vectorization.adapt(train_sinhala_texts)

##Preparing Datasets for the Translation Task

In [32]:
batch_size = 64  # Define batch size

def format_dataset(eng, sin):
    eng = source_vectorization(eng)  # Vectorize English texts
    sin = target_vectorization(sin)  # Vectorize Sinhala texts
    max_length = tf.maximum(tf.shape(eng)[1], tf.shape(sin)[1])  # Get the maximum sequence length
    eng = tf.pad(eng, [[0, 0], [0, max_length - tf.shape(eng)[1]]])[:, :max_length]  # Pad or truncate English sequences
    sin = tf.pad(sin, [[0, 0], [0, max_length - tf.shape(sin)[1]]])[:, :max_length]  # Pad or truncate Sinhala sequences
    return ({
        "english": eng[:, :-1],
        "sinhala": sin[:, :-1],
    }, sin[:, 1:])  # Return formatted dataset with English and Sinhala inputs, and shifted Sinhala targets

def make_dataset(pairs):
    eng_texts, sin_texts = zip(*pairs)  # Unzip English-Sinhala pairs
    eng_texts = list(eng_texts)  # Convert to list
    sin_texts = list(sin_texts)  # Convert to list
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sin_texts))  # Create dataset from tensors
    dataset = dataset.batch(batch_size)  # Batch the dataset
    dataset = dataset.map(format_dataset, num_parallel_calls=4)  # Map format_dataset function to the dataset
    return dataset.shuffle(2048).prefetch(16).cache()  # Shuffle, prefetch, and cache the dataset

# Create training and validation datasets
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

# Print shapes of inputs and targets from the first batch of training dataset
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['sinhala'].shape: {inputs['sinhala'].shape}")
    print(f"targets.shape: {targets.shape}")

# Print a sample from the training dataset
print(list(train_ds.as_numpy_iterator())[50])


inputs['english'].shape: (64, 20)
inputs['sinhala'].shape: (64, 20)
targets.shape: (64, 20)
({'english': array([[   51,    55,     0, ...,     0,     0,     0],
       [    2,    34,    13, ...,     0,     0,     0],
       [   17,  3769,    16, ...,     0,     0,     0],
       ...,
       [11443,     4,  2279, ...,     0,     0,     0],
       [  756, 12835,     3, ...,     0,     0,     0],
       [   97,    31,   102, ...,     0,     0,     0]]), 'sinhala': array([[    2,   201,  3467, ...,     0,     0,     0],
       [    2,     5,     9, ...,     0,     0,     0],
       [    2,    50,   639, ...,     0,     0,     0],
       ...,
       [    2, 11226,  4791, ...,     0,     0,     0],
       [    2,   631,  1359, ...,     0,     0,     0],
       [    2,   149,  2241, ...,     0,     0,     0]])}, array([[  201,  3467,     3, ...,     0,     0,     0],
       [    5,     9,   394, ...,     0,     0,     0],
       [   50,   639,  5005, ...,     0,     0,     0],
       ...,
   

#Transformer Model

##Transformer Encoder Implemented as a Subclassed Layer

In [16]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # Multi-head self-attention mechanism
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        # Feed-forward neural network layers
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        # Layer normalization for the two sub-layers
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        # Self-attention mechanism
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        # Add and normalize the self-attention output with the input
        proj_input = self.layernorm_1(inputs + attention_output)
        # Feed-forward network processing
        proj_output = self.dense_proj(proj_input)
        # Add and normalize the output with the input and self-attention output
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config


##The Transformer Decorder

In [17]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        # Multi-head self-attention mechanism for decoder input
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        # Multi-head attention mechanism for encoder-decoder attention
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        # Feed-forward neural network layers
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        # Layer normalization for the three sub-layers
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        # Create a causal attention mask to prevent attending to future tokens
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        # Create a causal mask for the decoder input
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            # Combine the input mask with the causal mask
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        # Self-attention mechanism for decoder input
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        # Add and normalize the self-attention output with the input
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        # Attention mechanism for encoder-decoder attention
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        # Add and normalize the output with the input and attention output
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        # Feed-forward network processing
        proj_output = self.dense_proj(attention_output_2)
        # Add and normalize the output with the attention output and projection output
        return self.layernorm_3(attention_output_2 + proj_output)


## Positional Encoding

In [18]:
# Positional embedding layer for incorporating positional information into token embeddings
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Embedding layer for token embeddings
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        # Embedding layer for positional embeddings
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        # Generate token embeddings
        embedded_tokens = self.token_embeddings(inputs)
        # Generate positional embeddings
        embedded_positions = self.position_embeddings(positions)
        # Add positional embeddings to token embeddings
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Create a mask to indicate valid tokens
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config


## End-to-End Transformer

In [19]:
# Define parameters for the Transformer model
embed_dim = 256  # Dimensionality of the token embeddings
dense_dim = 2048  # Dimensionality of the feed-forward layer in the transformer blocks
num_heads = 8  # Number of attention heads

# Define inputs for the encoder and decoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")  # Input sequence for the encoder (English)
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="sinhala")  # Input sequence for the decoder (Sinhala)

# Embedding and encoding for the encoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)  # Add positional embedding
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)  # Apply TransformerEncoder to encode the input sequence

# Embedding, decoding, and output generation for the decoder inputs
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)  # Add positional embedding
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)  # Apply TransformerDecoder to decode the input sequence
x = layers.Dropout(0.5)(x)  # Apply dropout regularization to prevent overfitting
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)  # Generate output probabilities using a dense layer with softmax activation

# Define the end-to-end Transformer model
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)  # Combine encoder and decoder inputs to create the Transformer model

# Print model summary
transformer.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 english (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 sinhala (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 256)            3845120   ['english[0][0]']             
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 256)            3845120   ['sinhala[0][0]']         

## Training the Sequence-to-Sequence Transformer

In [20]:
# Compile the Transformer model with RMSprop optimizer and sparse categorical crossentropy loss
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

# Train the Transformer model on the training dataset for 30 epochs with validation on the validation dataset
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
883/883 [==============================] - 76s 76ms/step - loss: 4.6927 - accuracy: 0.4016 - val_loss: 3.8734 - val_accuracy: 0.4613
Epoch 2/30
883/883 [==============================] - 59s 67ms/step - loss: 3.9180 - accuracy: 0.4669 - val_loss: 3.5535 - val_accuracy: 0.4907
Epoch 3/30
883/883 [==============================] - 60s 68ms/step - loss: 3.6122 - accuracy: 0.4950 - val_loss: 3.3709 - val_accuracy: 0.5121
Epoch 4/30
883/883 [==============================] - 60s 68ms/step - loss: 3.4253 - accuracy: 0.5157 - val_loss: 3.3506 - val_accuracy: 0.5142
Epoch 5/30
883/883 [==============================] - 64s 72ms/step - loss: 3.2902 - accuracy: 0.5329 - val_loss: 3.3820 - val_accuracy: 0.5116
Epoch 6/30
883/883 [==============================] - 64s 72ms/step - loss: 3.1904 - accuracy: 0.5477 - val_loss: 3.3034 - val_accuracy: 0.5239
Epoch 7/30
883/883 [==============================] - 60s 68ms/step - loss: 3.1094 - accuracy: 0.5595 - val_loss: 3.3625 - val_accuracy:

#Testing the Trained Model

##Testing the Transformer on Test Dataset

In [24]:
import numpy as np

# Get the vocabulary and create a lookup dictionary for the target language
sin_vocab = target_vectorization.get_vocabulary()
sin_index_lookup = dict(zip(range(len(sin_vocab)), sin_vocab))

# Define the maximum length for decoding a sentence
max_decoded_sentence_length = 20

# Function to decode a sequence given an input sentence
def decode_sequence(input_sentence):
    # Vectorize the input sentence
    tokenized_input_sentence = source_vectorization([input_sentence])

    # Initialize the decoded sentence with a start token
    decoded_sentence = "[start]"

    # Loop over the maximum decoded sentence length
    for i in range(max_decoded_sentence_length):
        # Vectorize the current decoded sentence (excluding the last token)
        tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]

        # Get predictions from the transformer model
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # Sample the token index with the highest probability
        sampled_token_index = np.argmax(predictions[0, i, :])

        # Get the corresponding token from the lookup dictionary
        sampled_token = sin_index_lookup[sampled_token_index]

        # Append the sampled token to the decoded sentence
        decoded_sentence += " " + sampled_token

        # Check if the end token is reached, and break the loop if so
        if sampled_token == "[end]":
            break

    return decoded_sentence

# Get a list of English sentences from the test pairs
test_eng_texts = [pair[0] for pair in test_pairs]

# Loop over a number of iterations to decode random sentences
for _ in range(20):
    # Choose a random input sentence from the list of English sentences
    input_sentence = random.choice(test_eng_texts)

    # Print the input sentence
    print("-")
    print("English : ", input_sentence)

    # Decode the input sentence and print the decoded sequence
    print("Sinhala : ", decode_sequence(input_sentence))


-
English :  i don't just talk to spirits
Sinhala :  [start] මට කියන්නේ නෑ එහෙම කතා කරන්න [end]
-
English :   said to let me go  don't let him get away
Sinhala :  [start] [UNK] යන්න දෙන්න කිව්වා එයාව අතහරින්න කියලා [end]
-
English :  ah
Sinhala :  [start] ආහ් [end]
-
English :  go
Sinhala :  [start] යන්න [end]
-
English :  i can't imagine how they found the person who made this
Sinhala :  [start] මට හිතාගන්න බෑ මේක කරපු විදිය දැනගත්තා මේ කවුද කියලා [end]
-
English :  what  
Sinhala :  [start] මොකක් [end]
-
English :  my turn
Sinhala :  [start] මගේ [UNK] [end]
-
English :  i
Sinhala :  [start] මම [end]
-
English :  damn
Sinhala :  [start] මගුල [end]
-
English :  a long time ago
Sinhala :  [start] ගොඩක් කාලයක් [end]
-
English :   come and come mumbaikars  mom
Sinhala :  [start] එන්න [UNK] එන්න අම්මා [end]
-
English :  when it went near my locker did it trip and fall into my locker  
Sinhala :  [start] මගේ [UNK] ඇවිත් මගේ [UNK] [UNK] [UNK] වුනත් [end]
-
English :   මගේ ආදරය දැන් මගේ නොවනු

## Testing the Transformer on User Inputs

In [31]:
# Get English input from the user
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence to Sinhala
translated_sentence = decode_sequence(input_sentence)

# Print the translated sentence
print("Sinhala translation:", translated_sentence)

Enter an English sentence: You are beautiful
Sinhala translation: [start] ඔයා ලස්සනයි [end]
